In [1]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QFileDialog, QWidget, QVBoxLayout, QHBoxLayout, QSpacerItem, QSizePolicy, QFrame, QDesktopWidget, QScrollArea
from PyQt5.QtCore import Qt, QRect, QPoint, QTimer, QFileInfo
from PyQt5.QtGui import QPainter, QPen, QImage, QPixmap, QFont
import cv2
import sys
import os
import numpy as np
from PIL import Image

In [2]:
def pil_imagem_qpixmap(image):
    """Converte uma imagem PIL para QPixmap"""
    image = image.convert("RGBA")
    date = image.tobytes("raw", "RGBA")
    qimage = QImage(date, image.width, image.height, QImage.Format_RGBA8888)
    pixmap = QPixmap.fromImage(qimage)
    return pixmap

In [3]:
def zoom(img, yi, yf, xi, xf):
    z = img[yi:yf, xi:xf]
    return z

In [4]:
# class JanelaSecundaria(QMainWindow):
#     def __init__(self, file_path, x, y, tipo_arquivo, janela_principal):
#         super().__init__()
#         self.setWindowTitle("Manipulação de Imagens e Vídeos")
#         self.tipo_arquivo = tipo_arquivo
#         self.janela_principal = janela_principal

#         self._setup_ui(file_path)

#     def _setup_ui(self, file_path):
#         self.image_label = QLabel(self)
#         self.image_label.setAlignment(Qt.AlignCenter)
#         # Configurações de layout
#         main_layout = QVBoxLayout()
#         main_layout.addWidget(self.image_label)
        
#         # Configuração do Scroll
#         self.scroll_area = QScrollArea(self)
#         self.scroll_area.setWidgetResizable(True)
#         self.scroll_area.setWidget(self.image_label)
#         main_layout.addWidget(self.scroll_area)

#         central_widget = QWidget()
#         central_widget.setLayout(main_layout)
#         self.setCentralWidget(central_widget)

#         # Carregar imagem
#         self.carregar_arquivo(file_path)

#     def carregar_arquivo(self, file_path):
#         if self.tipo_arquivo == "Imagem":
#             imagem = carregar_imagem(file_path)
#             if imagem is not None:
#                 pixmap = QPixmap.fromImage(imagem)
#                 self.image_label.setPixmap(pixmap)
#             else:
#                 print("Erro ao carregar a imagem.")

In [5]:
# def carregar_imagem(file_path):
#     imagem = cv2.imread(file_path)
#     if imagem is not None:
#         imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
#         altura, largura, canal = imagem.shape
#         bytes_por_linha = canal * largura
#         return QImage(imagem.data, largura, altura, bytes_por_linha, QImage.Format_RGB888)
#     return None

# def aplicar_filtro(imagem, tipo_filtro):
#     if tipo_filtro == "blur":
#         return cv2.GaussianBlur(imagem, (15, 15), 0)
#     elif tipo_filtro == "sharpen":
#         kernel = [[0, -1, 0], [-1, 5, -1], [0, -1, 0]]
#         return cv2.filter2D(imagem, -1, kernel)
#     # Outros filtros aqui
#     return imagem

# def redimensionar_imagem(imagem, escala):
#     altura, largura = imagem.shape[:2]
#     nova_dimensao = (int(largura * escala), int(altura * escala))
#     return cv2.resize(imagem, nova_dimensao)

In [ ]:
def _config_botoes(janela, layout):
        btn_layout = QHBoxLayout()
        btn_estilo = {
            "font-family": "Arial",
            "font-size": "15px",
            "color": "#000000",
            "background-color": "#d3d3d3",
            "padding": "5px 20px",
            "margin": "5px",
            "width": "10px",
            "height": "10px"
        }

        janela.btn_voltar_inf = janela._criar_btn("Voltar", btn_estilo, janela.voltar)
        janela.btn_zoom_mais = janela._criar_btn("Zoom+", btn_estilo, janela.zoom_mais)
        janela.btn_zoom_menos = janela._criar_btn("Zoom-", btn_estilo, janela.zoom_menos)
        janela.btn_aplicar_filtro = janela._criar_btn("Aplicar Filtro", btn_estilo, janela.aplicar_filtro_convolucao)
        janela.btn_desfaz_selecao = janela._criar_btn("Desfazer Seleção", btn_estilo)
        janela.btn_salvar = janela._criar_btn("Salvar", btn_estilo, self.salvar_recorte)
        janela.btn_sair = janela._criar_btn("Sair", btn_estilo, self.close)

        btn_layout.addWidget(janela.btn_voltar_inf)
        btn_layout.addWidget(janela.btn_zoom_mais)
        btn_layout.addWidget(janela.btn_zoom_menos)
        btn_layout.addWidget(janela.btn_aplicar_filtro)
        btn_layout.addWidget(janela.btn_desfaz_selecao)
        btn_layout.addWidget(janela.btn_salvar)
        btn_layout.addWidget(janela.btn_sair)

        layout.addLayout(btn_layout)

In [6]:
class JanelaSecundaria(QMainWindow):
    def __init__(self, file_path, x, y, tipo_arquivo, janela_principal):
        super().__init__()
        self.setWindowTitle("Manipulação de Imagens e Vídeos")
        self.setGeometry(100, 100, 1000, 800)
        self.tipo_arquivo = tipo_arquivo
        self.video_capture = None
        self.timer = QTimer()
        self.janela_principal = janela_principal
        self.original_image = None 
        self.cropped_image = None
        self.zoom_level = 1.0  
        self.rect_inicio_ponto = None 
        self.rect_fim_ponto = None
        self.roi_coords = []
        self.imagem_convolucao = None  # Armazena a imagem filtrada
        self._setup_ui(file_path, x, y)
        self.showMaximized()

    def _setup_ui(self, file_path, x, y):
        main_layout = QVBoxLayout()
        control_layout = QHBoxLayout()
        control_layout.setAlignment(Qt.AlignTop)
        self.image_label = QLabel(self)
        self.image_label.setAlignment(Qt.AlignCenter)

        # Seção "Aplicar Filtro"
        filtro_layout = QVBoxLayout()
        filtro_label = QLabel("Aplicar Filtro de Convolução", self)
        filtro_label.setFont(QFont("Arial", 12))
        filtro_layout.addWidget(filtro_label)
        self.filtro_combo = QComboBox(self)
        self.filtro_combo.addItems(["blur", "sharpen", "emboss", "laplacian", "canny", "sobel"])
        filtro_layout.addWidget(self.filtro_combo)
        control_layout.addLayout(filtro_layout)

        # Seção "Geração de Imagens"
        geracao_layout = QVBoxLayout()
        geracao_label = QLabel("Aplicar Fitro de Geração de Imagens", self)
        geracao_label.setFont(QFont("Arial", 12))
        geracao_layout.addWidget(geracao_label)
        self.geracao_combo = QComboBox(self)
        self.geracao_combo.addItems(["cinza", "binario", "cores"])
        geracao_layout.addWidget(self.geracao_combo)
        control_layout.addLayout(geracao_layout)

        # Seção "Vídeo"
        self.video_layout = QVBoxLayout()
        self.video_label = QLabel("Vídeo", self)
        self.video_label.setFont(QFont("Arial", 12))
        self.video_layout.addWidget(self.video_label)

        btn_layout = QHBoxLayout()
        self.btn_voltar = QPushButton("Voltar", self)
        self.btn_play = QPushButton("Play", self)
        self.btn_pause = QPushButton("Pause", self)
        self.btn_avancar = QPushButton("Avançar", self)
        self.btn_inverter = QPushButton("Inverter", self)
    
        btn_layout.addWidget(self.btn_voltar)
        btn_layout.addWidget(self.btn_play)
        btn_layout.addWidget(self.btn_pause)
        btn_layout.addWidget(self.btn_avancar)
        btn_layout.addWidget(self.btn_inverter)
        self.video_layout.addLayout(btn_layout)

        self.video_duracao = QLabel("00:00 / 00:00", self)
        self.video_layout.addWidget(self.video_duracao)
        control_layout.addLayout(self.video_layout)

        # Área de visualização
        self.image_label = QLabel(self)
        self.scroll_area = QScrollArea(self)
        self.scroll_area.setFixedSize(700, 500)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.scroll_area.setWidgetResizable(True)
        self.scroll_area.setWidget(self.image_label)

        # Remover barras de rolagem para vídeos
        if self.tipo_arquivo == "Vídeo":
            self.scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
            self.scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
            self.scroll_area.setAlignment(Qt.AlignCenter)
        else:
            self.video_label.setVisible(False)
            self.btn_voltar.setVisible(False)
            self.btn_play.setVisible(False)
            self.btn_pause.setVisible(False)
            self.btn_avancar.setVisible(False)
            self.btn_inverter.setVisible(False)
            self.video_duracao.setVisible(False)
    
        # Layout de centralização
        center_layout = QVBoxLayout()
        center_layout.addStretch(1)
        center_layout.addWidget(self.scroll_area, alignment=Qt.AlignCenter)
        center_layout.addStretch(1)

        main_layout.addLayout(control_layout)
        main_layout.addLayout(center_layout)

        print(f"Carregando imagem do caminho: {file_path}")
        if self.tipo_arquivo == "Imagem":
            self.original_image = cv2.imread(file_path)
            if self.original_image is not None: 
                print("Imagem carregada com sucesso.") 
                self.carregar_imagem()
            else: 
                print("Falha ao carregar a imagem.")
        
        elif self.tipo_arquivo == "Vídeo":
            self.video_capture = cv2.VideoCapture(file_path)
            self.timer.timeout.connect(self.atualizar_quadro)
            self.timer.start(30) 

        central_widget = QWidget()
        central_widget.setLayout(main_layout)
        self.setCentralWidget(central_widget)
        self.setStyleSheet("background-color: #e6f2ff;")
    
        self._config_botoes(main_layout)

    def opencv_imagem_qpixmap(self, image):
        altura, largura, canal = image.shape
        bytes_por_linha = canal * largura
        qimage = QImage(image.data, largura, altura, bytes_por_linha, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        return pixmap

    def carregar_imagem(self):
        if self.original_image is not None:
            imagem = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
            pixmap = self.opencv_imagem_qpixmap(imagem)
            self.image_label.setPixmap(pixmap.scaled(1000, 500, Qt.KeepAspectRatio, Qt.SmoothTransformation))
            self.image_label.resize(1000, 500)

    def concluir_selecao(self):
        if self.selecionando:
            self.selecionando = False
            x1, y1 = min(self.start_x, self.end_x), min(self.start_y, self.end_y)
            x2, y2 = max(self.start_x, self.end_x), max(self.start_y, self.end_y)
        
            # Adicionando logs para depuração
            print(f"Initial selection coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
        
            # Ajustar as coordenadas considerando a proporção de zoom
            proporcao_x = self.original_image.shape[1] / self.image_label.pixmap().width()
            proporcao_y = self.original_image.shape[0] / self.image_label.pixmap().height()
        
            x1 = int(x1 * proporcao_x)
            y1 = int(y1 * proporcao_y)
            x2 = int(x2 * proporcao_x)
            y2 = int(y2 * proporcao_y)
        
            # Adicionando logs para depuração
            print(f"Scaled selection coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
        
            if self.original_image is not None:
                self.cropped_image = self.original_image[y1:y2, x1:x2]
                self.exibir_imagem(self.cropped_image)
                print("Seleção concluída e imagem recortada armazenada.")
            else:
                self.cropped_image = None
                print("Erro ao concluir a seleção: imagem original não encontrada.")


    def exibir_imagem(self, image):
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pixmap = self.opencv_imagem_qpixmap(rgb_image)
        self.image_label.setPixmap(pixmap.scaled(1000, 500, Qt.KeepAspectRatio, Qt.SmoothTransformation))


    def atualizar_quadro(self):
        ret, frame = self.video_capture.read()
        if ret:
            imagem = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            altura, largura, canal = imagem.shape
            passo = canal * largura
            qimage = QImage(imagem.data, largura, altura, passo, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qimage)

            self.image_label.setPixmap(pixmap.scaled(self.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            self.image_label.setFixedSize(self.scroll_area.size())
            self.scroll_area.setWidget(self.image_label)
        else:
            self.timer.stop()
            self.video_capture.release()

    def closeEvent(self, event):
        if self.video_capture is not None:
            self.video_capture.release()
        super().closeEvent(event)

    def voltar(self):
        self.close()
        self.janela_principal.show()

    config

    

    def _config_filtro(self):
    # Adicionar um método para configurar os filtros de convolução
        self.btn_filtro.clicked.connect(self.aplicar_filtro_convolucao)

    def aplicar_filtro_convolucao(self):
        if self.original_image is not None:
            filtro_selecionado = self.filtro_combo.currentText()
            try:
                filtered_image = self.aplicar_filtro(self.original_image, filtro_selecionado)
                filtered_image_cv = np.array(filtered_image)
            
                if filtro_selecionado in ["laplacian", "sobel", "scharr", "prewitt"]:
                    filtered_image_cv = cv2.normalize(filtered_image_cv, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            
                self.exibir_imagem(filtered_image_cv)
                self.original_image = filtered_image_cv
                self.imagem_convolucao = filtered_image_cv
            
                print(f"Filtro {filtro_selecionado} aplicado com sucesso.")
            except Exception as e:
                print(f"Erro ao aplicar filtro: {e}")
        else:
            print("Nenhuma imagem carregada para aplicar filtro.")

    def aplicar_filtro(self, image, filter_type):
        cv_image = np.array(image)
    
        if filter_type == "blur":
            print("Aplicando filtro: blur")
            filtered_image = cv2.GaussianBlur(cv_image, (15, 15), 0)
        elif filter_type == "sharpen":
            print("Aplicando filtro: sharpen")
            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
            filtered_image = cv2.filter2D(cv_image, -1, kernel)
        elif filter_type == "emboss":
            print("Aplicando filtro: emboss")
            kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
            filtered_image = cv2.filter2D(cv_image, -1, kernel)
        elif filter_type == "laplacian":
            print("Aplicando filtro: laplacian")
            filtered_image = cv2.Laplacian(cv_image, cv2.CV_64F)
        elif filter_type == "canny":
            print("Aplicando filtro: canny")
            filtered_image = cv2.Canny(cv_image, 100, 200)
        elif filter_type == "sobel":
            print("Aplicando filtro: sobel")
            filtered_image = cv2.Sobel(cv_image, cv2.CV_64F, 1, 0, ksize=5)
        elif filter_type == "median_blur":
            print("Aplicando filtro: median_blur")
            filtered_image = cv2.medianBlur(cv_image, 5)
        elif filter_type == "bilateral_filter":
            print("Aplicando filtro: bilateral_filter")
            filtered_image = cv2.bilateralFilter(cv_image, 9, 75, 75)
        elif filter_type == "scharr":
            print("Aplicando filtro: scharr")
            filtered_image = cv2.Scharr(cv_image, cv2.CV_64F, 1, 0)
        elif filter_type == "prewitt":
            print("Aplicando filtro: prewitt")
            kernelx = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
            kernely = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
            filtered_image_x = cv2.filter2D(cv_image, -1, kernelx)
            filtered_image_y = cv2.filter2D(cv_image, -1, kernely)
            filtered_image = cv2.addWeighted(filtered_image_x, 0.5, filtered_image_y, 0.5, 0)
        else:
            filtered_image = cv_image

        print(f"Filtro {filter_type} aplicado.")
        return filtered_image

    def _criar_btn(self, texto, estilo, chamada_retorno=None):
        btn = QPushButton(texto, self)
        btn.setStyleSheet(f"""
            font-family: {estilo['font-family']};
            font-size: {estilo['font-size']};
            color: {estilo['color']};
            background-color: {estilo['background-color']};
            padding:{estilo['padding']};
            margin: {estilo['margin']};
            width: {estilo['width']};
            height: {estilo['height']};
        """)
        btn.setFont(QFont(estilo['font-family'], int(estilo['font-size'].replace('px', ''))))
        if chamada_retorno:
            btn.clicked.connect(chamada_retorno)
        return btn

    def zoom_mais(self):
        if self.original_image is not None:
            print(f"Zoom+: Level Before: {self.zoom_level}")
            self.zoom_level = min(self.zoom_level * 1.2, 4.0)  # Mantemos um fator de 1.2 para aumento
            self.aplicar_zoom_customizado()
    
    def zoom_menos(self):
        if self.original_image is not None:
            print(f"Zoom-: Level Before: {self.zoom_level}")
            self.zoom_level = max(self.zoom_level / 1.2, 1.0)  # Mesmo fator de 1.2 para diminuição
            if self.zoom_level <= 1.01:  # Pequena margem para garantir que volte ao original
                self.zoom_level = 1.0
                self.aplicar_tamanho_original()
            else:
                self.aplicar_zoom_customizado()

    def aplicar_tamanho_original(self):
        if self.original_image is not None:
            try:
                original_img_rgb = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
                pixmap = self.opencv_imagem_qpixmap(original_img_rgb)
                self.image_label.setPixmap(pixmap.scaled(self.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
                self.image_label.resize(self.scroll_area.size())
                self.scroll_area.setWidget(self.image_label)
                self.scroll_area.setWidgetResizable(True)
                print("Imagem redefinida para o tamanho original.")
            except Exception as e:
                print(f"Erro ao redefinir para o tamanho original: {e}")
        else:
            print("Nenhuma imagem carregada para redefinir.") 

    def aplicar_zoom_customizado(self):
        if self.original_image is not None:
            try:
                altura, largura, _ = self.original_image.shape
                nova_altura = int(altura * self.zoom_level)
                nova_largura = int(largura * self.zoom_level)
                zoomed_img = cv2.resize(self.original_image, (nova_largura, nova_altura), interpolation=cv2.INTER_LINEAR)
                zoomed_img_rgb = cv2.cvtColor(zoomed_img, cv2.COLOR_BGR2RGB)
                pixmap = self.opencv_imagem_qpixmap(zoomed_img_rgb)
                self.image_label.setPixmap(pixmap)
                self.image_label.resize(nova_largura, nova_altura)
                self.scroll_area.setWidget(self.image_label)
                self.scroll_area.setWidgetResizable(True)
                self.scroll_area.setFixedSize(1000, 500)
                print(f"Zoom aplicado: {nova_largura}x{nova_altura}")
            except Exception as e:
                print(f"Erro ao aplicar zoom: {e}")
        else:
            print("Nenhuma imagem carregada para aplicar zoom")

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.rect_inicio_ponto = event.pos()
            self.rect_fim_ponto = None

    def mouseMoveEvent(self, event):
        if self.rect_inicio_ponto:
            self.rect_fim_ponto = event.pos()
            self.update()

    def mouse_release_event(self, event):
        if event.button() == Qt.LeftButton:
            if self.selecionando:
                self.end_x = event.pos().x()
                self.end_y = event.pos().y()
                print(f"Mouse released at: {self.end_x}, {self.end_y}")  # Log para depuração
                self.atualizar_selecao()
                self.concluir_selecao()  # Certifique-se de concluir a seleção

    def atualizar_selecao(self):
        if self.original_image is not None and self.selecionando:
            temp_image = self.original_image.copy()
            x1, y1 = self.start_x, self.start_y
            x2, y2 = self.end_x, self.end_y
            # Certifique-se de que as coordenadas estejam dentro dos limites
            x1, x2 = max(0, x1), min(temp_image.shape[1], x2)
            y1, y2 = max(0, y1), min(temp_image.shape[0], y2)
            # Desenhar o retângulo vermelho
            cv2.rectangle(temp_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            self.exibir_imagem(temp_image)
            print(f"Drawing rectangle from ({x1}, {y1}) to ({x2}, {y2})")  # Log para depuração
        else:
            print("Erro: Coordenadas inválidas ou imagem não carregada.")

    def salvar_recorte(self):
        if self.rect_inicio_ponto and self.rect_fim_ponto:
            # Posição do scroll
            scroll_x = self.scroll_area.horizontalScrollBar().value()
            scroll_y = self.scroll_area.verticalScrollBar().value()

            # Coordenadas de seleção ajustadas pela posição do scroll
            x1, y1 = self.rect_inicio_ponto.x() + scroll_x, self.rect_inicio_ponto.y() + scroll_y
            x2, y2 = self.rect_fim_ponto.x() + scroll_x, self.rect_fim_ponto.y() + scroll_y
        
            x1, x2 = sorted([x1, x2])
            y1, y2 = sorted([y1, y2])

            print(f"Before scaling: x1={x1}, y1={y1}, x2={x2}, y2={y2}")  # Log para depuração
        
            # Proporção de escala da imagem original em relação ao tamanho da image_label
            proporcao_x = self.original_image.shape[1] / self.image_label.pixmap().width()
            proporcao_y = self.original_image.shape[0] / self.image_label.pixmap().height()
        
            x1 = int(x1 * proporcao_x)
            y1 = int(y1 * proporcao_y)
            x2 = int(x2 * proporcao_x)
            y2 = int(y2 * proporcao_y)

            print(f"After scaling: x1={x1}, y1={y1}, x2={x2}, y2={y2}")  # Log para depuração
        
            recorte = self.original_image[y1:y2, x1:x2]
        
            # Abrir um diálogo para selecionar o diretório de salvamento
            path_salvar = QFileDialog.getExistingDirectory(self, "C:/Users/wallace/Desktop")
            if path_salvar:
                file_path = f"{path_salvar}/frame_alterado.jpg"
                cv2.imwrite(file_path, recorte)
                print(f"Recorte salvo em: {file_path}")
        
            self.rect_inicio_ponto = None
            self.rect_fim_ponto = None
            self.update()
        else:
            print("Nenhuma área recortada para salvar.") 

        # Salvando a imagem convolucionada (com filtro aplicado)
        if self.imagem_convolucao is not None:
            try:
                path_salvar_filtro = QFileDialog.getExistingDirectory(self, "C:")
                if path_salvar_filtro:
                    file_path_filtro = f"{path_salvar_filtro}/imagem_com_filtro.jpg"
                    cv2.imwrite(file_path_filtro, self.imagem_convolucao)
                    print(f"Imagem com filtro salva em: {file_path_filtro}")
            except Exception as e:
                print(f"Erro ao salvar a imagem com filtro: {e}")
        else:
            print("Nenhuma imagem com filtro para salvar.")

    def paintEvent(self, event):
        super().paintEvent(event)
        if self.rect_inicio_ponto and self.rect_fim_ponto:
            painter = QPainter(self)
            painter.setPen(QPen(Qt.red, 2, Qt.SolidLine))
            rect = QRect(self.rect_inicio_ponto, self.rect_fim_ponto)
            painter.drawRect(rect.normalized())

In [7]:
# class JanelaSecundaria(QMainWindow):
#     def __init__(self, file_path, x, y, tipo_arquivo, janela_principal):
#         super().__init__()
#         self.setWindowTitle("Manipulação de Imagens e Vídeos")
#         self.setGeometry(100, 100, 1000, 800)
#         self.tipo_arquivo = tipo_arquivo
#         self.video_capture = None
#         self.timer = QTimer()
#         self.janela_principal = janela_principal
#         self.original_image = None 
#         self.cropped_image = None
#         self.zoom_level = 1.0  
#         self.rect_inicio_ponto = None 
#         self.rect_fim_ponto = None
#         self.roi_coords = []
#         self.imagem_convolucao = None  # Armazena a imagem filtrada
#         self._setup_ui(file_path, x, y)
#         self.showMaximized()

#     def _setup_ui(self, file_path, x, y):
#         main_layout = QVBoxLayout()
#         control_layout = QHBoxLayout()
#         control_layout.setAlignment(Qt.AlignTop)
#         self.image_label = QLabel(self)
#         self.image_label.setAlignment(Qt.AlignCenter)

#         # Seção "Aplicar Filtro"
#         filtro_layout = QVBoxLayout()
#         filtro_label = QLabel("Aplicar Filtro de Convolução", self)
#         filtro_label.setFont(QFont("Arial", 12))
#         filtro_layout.addWidget(filtro_label)
#         self.filtro_combo = QComboBox(self)
#         self.filtro_combo.addItems(["blur", "sharpen", "emboss", "laplacian", "canny", "sobel"])
#         filtro_layout.addWidget(self.filtro_combo)
#         control_layout.addLayout(filtro_layout)

#         # Seção "Geração de Imagens"
#         geracao_layout = QVBoxLayout()
#         geracao_label = QLabel("Aplicar Fitro de Geração de Imagens", self)
#         geracao_label.setFont(QFont("Arial", 12))
#         geracao_layout.addWidget(geracao_label)
#         self.geracao_combo = QComboBox(self)
#         self.geracao_combo.addItems(["cinza", "binario", "cores"])
#         geracao_layout.addWidget(self.geracao_combo)
#         control_layout.addLayout(geracao_layout)

#         # Seção "Vídeo"
#         self.video_layout = QVBoxLayout()
#         self.video_label = QLabel("Vídeo", self)
#         self.video_label.setFont(QFont("Arial", 12))
#         self.video_layout.addWidget(self.video_label)

#         btn_layout = QHBoxLayout()
#         self.btn_voltar = QPushButton("Voltar", self)
#         self.btn_play = QPushButton("Play", self)
#         self.btn_pause = QPushButton("Pause", self)
#         self.btn_avancar = QPushButton("Avançar", self)
#         self.btn_inverter = QPushButton("Inverter", self)
    
#         btn_layout.addWidget(self.btn_voltar)
#         btn_layout.addWidget(self.btn_play)
#         btn_layout.addWidget(self.btn_pause)
#         btn_layout.addWidget(self.btn_avancar)
#         btn_layout.addWidget(self.btn_inverter)
#         self.video_layout.addLayout(btn_layout)

#         self.video_duracao = QLabel("00:00 / 00:00", self)
#         self.video_layout.addWidget(self.video_duracao)
#         control_layout.addLayout(self.video_layout)

#         # Área de visualização
#         self.image_label = QLabel(self)
#         self.scroll_area = QScrollArea(self)
#         self.scroll_area.setFixedSize(700, 500)
#         self.image_label.setAlignment(Qt.AlignCenter)
#         self.scroll_area.setWidgetResizable(True)
#         self.scroll_area.setWidget(self.image_label)

#         # Remover barras de rolagem para vídeos
#         if self.tipo_arquivo == "Vídeo":
#             self.scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
#             self.scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
#             self.scroll_area.setAlignment(Qt.AlignCenter)
#         else:
#             self.video_label.setVisible(False)
#             self.btn_voltar.setVisible(False)
#             self.btn_play.setVisible(False)
#             self.btn_pause.setVisible(False)
#             self.btn_avancar.setVisible(False)
#             self.btn_inverter.setVisible(False)
#             self.video_duracao.setVisible(False)
    
#         # Layout de centralização
#         center_layout = QVBoxLayout()
#         center_layout.addStretch(1)
#         center_layout.addWidget(self.scroll_area, alignment=Qt.AlignCenter)
#         center_layout.addStretch(1)

#         main_layout.addLayout(control_layout)
#         main_layout.addLayout(center_layout)

#         print(f"Carregando imagem do caminho: {file_path}")
#         if self.tipo_arquivo == "Imagem":
#             self.original_image = cv2.imread(file_path)
#             if self.original_image is not None: 
#                 print("Imagem carregada com sucesso.") 
#                 self.carregar_imagem()
#             else: 
#                 print("Falha ao carregar a imagem.")
        
#         elif self.tipo_arquivo == "Vídeo":
#             self.video_capture = cv2.VideoCapture(file_path)
#             self.timer.timeout.connect(self.atualizar_quadro)
#             self.timer.start(30) 

#         central_widget = QWidget()
#         central_widget.setLayout(main_layout)
#         self.setCentralWidget(central_widget)
#         self.setStyleSheet("background-color: #e6f2ff;")
    
#         self._config_botoes(main_layout)

#     def opencv_imagem_qpixmap(self, image):
#         altura, largura, canal = image.shape
#         bytes_por_linha = canal * largura
#         qimage = QImage(image.data, largura, altura, bytes_por_linha, QImage.Format_RGB888)
#         pixmap = QPixmap.fromImage(qimage)
#         return pixmap

#     def carregar_imagem(self):
#         if self.original_image is not None:
#             imagem = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
#             pixmap = self.opencv_imagem_qpixmap(imagem)
#             self.image_label.setPixmap(pixmap.scaled(1000, 500, Qt.KeepAspectRatio, Qt.SmoothTransformation))
#             self.image_label.resize(1000, 500)

#     def concluir_selecao(self):
#         if self.selecionando:
#             self.selecionando = False
#             x1, y1 = min(self.start_x, self.end_x), min(self.start_y, self.end_y)
#             x2, y2 = max(self.start_x, self.end_x), max(self.start_y, self.end_y)
        
#             # Adicionando logs para depuração
#             print(f"Initial selection coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
        
#             # Ajustar as coordenadas considerando a proporção de zoom
#             proporcao_x = self.original_image.shape[1] / self.image_label.pixmap().width()
#             proporcao_y = self.original_image.shape[0] / self.image_label.pixmap().height()
        
#             x1 = int(x1 * proporcao_x)
#             y1 = int(y1 * proporcao_y)
#             x2 = int(x2 * proporcao_x)
#             y2 = int(y2 * proporcao_y)
        
#             # Adicionando logs para depuração
#             print(f"Scaled selection coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
        
#             if self.original_image is not None:
#                 self.cropped_image = self.original_image[y1:y2, x1:x2]
#                 self.exibir_imagem(self.cropped_image)
#                 print("Seleção concluída e imagem recortada armazenada.")
#             else:
#                 self.cropped_image = None
#                 print("Erro ao concluir a seleção: imagem original não encontrada.")


#     def exibir_imagem(self, image):
#         rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         pixmap = self.opencv_imagem_qpixmap(rgb_image)
#         self.image_label.setPixmap(pixmap.scaled(1000, 500, Qt.KeepAspectRatio, Qt.SmoothTransformation))


#     def atualizar_quadro(self):
#         ret, frame = self.video_capture.read()
#         if ret:
#             imagem = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             altura, largura, canal = imagem.shape
#             passo = canal * largura
#             qimage = QImage(imagem.data, largura, altura, passo, QImage.Format_RGB888)
#             pixmap = QPixmap.fromImage(qimage)

#             self.image_label.setPixmap(pixmap.scaled(self.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
#             self.image_label.setFixedSize(self.scroll_area.size())
#             self.scroll_area.setWidget(self.image_label)
#         else:
#             self.timer.stop()
#             self.video_capture.release()

#     def closeEvent(self, event):
#         if self.video_capture is not None:
#             self.video_capture.release()
#         super().closeEvent(event)

#     def voltar(self):
#         self.close()
#         self.janela_principal.show()

#     def _config_botoes(self, layout):
#         btn_layout = QHBoxLayout()
#         btn_estilo = {
#             "font-family": "Arial",
#             "font-size": "15px",
#             "color": "#000000",
#             "background-color": "#007acc",
#             "padding": "5px 20px",
#             "margin": "5px",
#             "width": "10px",
#             "height": "10px"
#         }

#         self.btn_voltar_inf = self._criar_btn("Voltar", btn_estilo, self.voltar)
#         self.btn_zoom_mais = self._criar_btn("Zoom+", btn_estilo, self.zoom_mais)
#         self.btn_zoom_menos = self._criar_btn("Zoom-", btn_estilo, self.zoom_menos)
#         self.btn_aplicar_filtro = self._criar_btn("Aplicar Filtro", btn_estilo, self.aplicar_filtro_convolucao)
#         self.btn_desfaz_selecao = self._criar_btn("Desfazer Seleção", btn_estilo)
#         self.btn_salvar = self._criar_btn("Salvar", btn_estilo, self.salvar_recorte)
#         self.btn_sair = self._criar_btn("Sair", btn_estilo, self.close)

#         btn_layout.addWidget(self.btn_voltar_inf)
#         btn_layout.addWidget(self.btn_zoom_mais)
#         btn_layout.addWidget(self.btn_zoom_menos)
#         btn_layout.addWidget(self.btn_aplicar_filtro)
#         btn_layout.addWidget(self.btn_desfaz_selecao)
#         btn_layout.addWidget(self.btn_salvar)
#         btn_layout.addWidget(self.btn_sair)

#         layout.addLayout(btn_layout)

#     def _config_filtro(self):
#     # Adicionar um método para configurar os filtros de convolução
#         self.btn_filtro.clicked.connect(self.aplicar_filtro_convolucao)

#     def aplicar_filtro_convolucao(self):
#         if self.original_image is not None:
#             filtro_selecionado = self.filtro_combo.currentText()
#             try:
#                 filtered_image = self.aplicar_filtro(self.original_image, filtro_selecionado)
#                 filtered_image_cv = np.array(filtered_image)
            
#                 if filtro_selecionado in ["laplacian", "sobel", "scharr", "prewitt"]:
#                     filtered_image_cv = cv2.normalize(filtered_image_cv, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            
#                 self.exibir_imagem(filtered_image_cv)
#                 self.original_image = filtered_image_cv
#                 self.imagem_convolucao = filtered_image_cv
            
#                 print(f"Filtro {filtro_selecionado} aplicado com sucesso.")
#             except Exception as e:
#                 print(f"Erro ao aplicar filtro: {e}")
#         else:
#             print("Nenhuma imagem carregada para aplicar filtro.")

#     def aplicar_filtro(self, image, filter_type):
#         cv_image = np.array(image)
    
#         if filter_type == "blur":
#             print("Aplicando filtro: blur")
#             filtered_image = cv2.GaussianBlur(cv_image, (15, 15), 0)
#         elif filter_type == "sharpen":
#             print("Aplicando filtro: sharpen")
#             kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
#             filtered_image = cv2.filter2D(cv_image, -1, kernel)
#         elif filter_type == "emboss":
#             print("Aplicando filtro: emboss")
#             kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
#             filtered_image = cv2.filter2D(cv_image, -1, kernel)
#         elif filter_type == "laplacian":
#             print("Aplicando filtro: laplacian")
#             filtered_image = cv2.Laplacian(cv_image, cv2.CV_64F)
#         elif filter_type == "canny":
#             print("Aplicando filtro: canny")
#             filtered_image = cv2.Canny(cv_image, 100, 200)
#         elif filter_type == "sobel":
#             print("Aplicando filtro: sobel")
#             filtered_image = cv2.Sobel(cv_image, cv2.CV_64F, 1, 0, ksize=5)
#         elif filter_type == "median_blur":
#             print("Aplicando filtro: median_blur")
#             filtered_image = cv2.medianBlur(cv_image, 5)
#         elif filter_type == "bilateral_filter":
#             print("Aplicando filtro: bilateral_filter")
#             filtered_image = cv2.bilateralFilter(cv_image, 9, 75, 75)
#         elif filter_type == "scharr":
#             print("Aplicando filtro: scharr")
#             filtered_image = cv2.Scharr(cv_image, cv2.CV_64F, 1, 0)
#         elif filter_type == "prewitt":
#             print("Aplicando filtro: prewitt")
#             kernelx = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
#             kernely = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
#             filtered_image_x = cv2.filter2D(cv_image, -1, kernelx)
#             filtered_image_y = cv2.filter2D(cv_image, -1, kernely)
#             filtered_image = cv2.addWeighted(filtered_image_x, 0.5, filtered_image_y, 0.5, 0)
#         else:
#             filtered_image = cv_image

#         print(f"Filtro {filter_type} aplicado.")
#         return filtered_image

#     def _criar_btn(self, texto, estilo, chamada_retorno=None):
#         btn = QPushButton(texto, self)
#         btn.setStyleSheet(f"""
#             font-family: {estilo['font-family']};
#             font-size: {estilo['font-size']};
#             color: {estilo['color']};
#             background-color: {estilo['background-color']};
#             padding:{estilo['padding']};
#             margin: {estilo['margin']};
#             width: {estilo['width']};
#             height: {estilo['height']};
#         """)
#         btn.setFont(QFont(estilo['font-family'], int(estilo['font-size'].replace('px', ''))))
#         if chamada_retorno:
#             btn.clicked.connect(chamada_retorno)
#         return btn

#     def zoom_mais(self):
#         if self.original_image is not None:
#             print(f"Zoom+: Level Before: {self.zoom_level}")
#             self.zoom_level = min(self.zoom_level * 1.2, 4.0)  # Mantemos um fator de 1.2 para aumento
#             self.aplicar_zoom_customizado()
    
#     def zoom_menos(self):
#         if self.original_image is not None:
#             print(f"Zoom-: Level Before: {self.zoom_level}")
#             self.zoom_level = max(self.zoom_level / 1.2, 1.0)  # Mesmo fator de 1.2 para diminuição
#             if self.zoom_level <= 1.01:  # Pequena margem para garantir que volte ao original
#                 self.zoom_level = 1.0
#                 self.aplicar_tamanho_original()
#             else:
#                 self.aplicar_zoom_customizado()

#     def aplicar_tamanho_original(self):
#         if self.original_image is not None:
#             try:
#                 original_img_rgb = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
#                 pixmap = self.opencv_imagem_qpixmap(original_img_rgb)
#                 self.image_label.setPixmap(pixmap.scaled(self.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
#                 self.image_label.resize(self.scroll_area.size())
#                 self.scroll_area.setWidget(self.image_label)
#                 self.scroll_area.setWidgetResizable(True)
#                 print("Imagem redefinida para o tamanho original.")
#             except Exception as e:
#                 print(f"Erro ao redefinir para o tamanho original: {e}")
#         else:
#             print("Nenhuma imagem carregada para redefinir.") 

#     def aplicar_zoom_customizado(self):
#         if self.original_image is not None:
#             try:
#                 altura, largura, _ = self.original_image.shape
#                 nova_altura = int(altura * self.zoom_level)
#                 nova_largura = int(largura * self.zoom_level)
#                 zoomed_img = cv2.resize(self.original_image, (nova_largura, nova_altura), interpolation=cv2.INTER_LINEAR)
#                 zoomed_img_rgb = cv2.cvtColor(zoomed_img, cv2.COLOR_BGR2RGB)
#                 pixmap = self.opencv_imagem_qpixmap(zoomed_img_rgb)
#                 self.image_label.setPixmap(pixmap)
#                 self.image_label.resize(nova_largura, nova_altura)
#                 self.scroll_area.setWidget(self.image_label)
#                 self.scroll_area.setWidgetResizable(True)
#                 self.scroll_area.setFixedSize(1000, 500)
#                 print(f"Zoom aplicado: {nova_largura}x{nova_altura}")
#             except Exception as e:
#                 print(f"Erro ao aplicar zoom: {e}")
#         else:
#             print("Nenhuma imagem carregada para aplicar zoom")

#     def mousePressEvent(self, event):
#         if event.button() == Qt.LeftButton:
#             self.rect_inicio_ponto = event.pos()
#             self.rect_fim_ponto = None

#     def mouseMoveEvent(self, event):
#         if self.rect_inicio_ponto:
#             self.rect_fim_ponto = event.pos()
#             self.update()

#     def mouse_release_event(self, event):
#         if event.button() == Qt.LeftButton:
#             if self.selecionando:
#                 self.end_x = event.pos().x()
#                 self.end_y = event.pos().y()
#                 print(f"Mouse released at: {self.end_x}, {self.end_y}")  # Log para depuração
#                 self.atualizar_selecao()
#                 self.concluir_selecao()  # Certifique-se de concluir a seleção

#     def atualizar_selecao(self):
#         if self.original_image is not None and self.selecionando:
#             temp_image = self.original_image.copy()
#             x1, y1 = self.start_x, self.start_y
#             x2, y2 = self.end_x, self.end_y
#             # Certifique-se de que as coordenadas estejam dentro dos limites
#             x1, x2 = max(0, x1), min(temp_image.shape[1], x2)
#             y1, y2 = max(0, y1), min(temp_image.shape[0], y2)
#             # Desenhar o retângulo vermelho
#             cv2.rectangle(temp_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
#             self.exibir_imagem(temp_image)
#             print(f"Drawing rectangle from ({x1}, {y1}) to ({x2}, {y2})")  # Log para depuração
#         else:
#             print("Erro: Coordenadas inválidas ou imagem não carregada.")

#     def salvar_recorte(self):
#         if self.rect_inicio_ponto and self.rect_fim_ponto:
#             # Posição do scroll
#             scroll_x = self.scroll_area.horizontalScrollBar().value()
#             scroll_y = self.scroll_area.verticalScrollBar().value()

#             # Coordenadas de seleção ajustadas pela posição do scroll
#             x1, y1 = self.rect_inicio_ponto.x() + scroll_x, self.rect_inicio_ponto.y() + scroll_y
#             x2, y2 = self.rect_fim_ponto.x() + scroll_x, self.rect_fim_ponto.y() + scroll_y
        
#             x1, x2 = sorted([x1, x2])
#             y1, y2 = sorted([y1, y2])

#             print(f"Before scaling: x1={x1}, y1={y1}, x2={x2}, y2={y2}")  # Log para depuração
        
#             # Proporção de escala da imagem original em relação ao tamanho da image_label
#             proporcao_x = self.original_image.shape[1] / self.image_label.pixmap().width()
#             proporcao_y = self.original_image.shape[0] / self.image_label.pixmap().height()
        
#             x1 = int(x1 * proporcao_x)
#             y1 = int(y1 * proporcao_y)
#             x2 = int(x2 * proporcao_x)
#             y2 = int(y2 * proporcao_y)

#             print(f"After scaling: x1={x1}, y1={y1}, x2={x2}, y2={y2}")  # Log para depuração
        
#             recorte = self.original_image[y1:y2, x1:x2]
        
#             # Abrir um diálogo para selecionar o diretório de salvamento
#             path_salvar = QFileDialog.getExistingDirectory(self, "C:/Users/wallace/Desktop")
#             if path_salvar:
#                 file_path = f"{path_salvar}/frame_alterado.jpg"
#                 cv2.imwrite(file_path, recorte)
#                 print(f"Recorte salvo em: {file_path}")
        
#             self.rect_inicio_ponto = None
#             self.rect_fim_ponto = None
#             self.update()
#         else:
#             print("Nenhuma área recortada para salvar.") 

#         # Salvando a imagem convolucionada (com filtro aplicado)
#         if self.imagem_convolucao is not None:
#             try:
#                 path_salvar_filtro = QFileDialog.getExistingDirectory(self, "C:")
#                 if path_salvar_filtro:
#                     file_path_filtro = f"{path_salvar_filtro}/imagem_com_filtro.jpg"
#                     cv2.imwrite(file_path_filtro, self.imagem_convolucao)
#                     print(f"Imagem com filtro salva em: {file_path_filtro}")
#             except Exception as e:
#                 print(f"Erro ao salvar a imagem com filtro: {e}")
#         else:
#             print("Nenhuma imagem com filtro para salvar.")

#     def paintEvent(self, event):
#         super().paintEvent(event)
#         if self.rect_inicio_ponto and self.rect_fim_ponto:
#             painter = QPainter(self)
#             painter.setPen(QPen(Qt.red, 2, Qt.SolidLine))
#             rect = QRect(self.rect_inicio_ponto, self.rect_fim_ponto)
#             painter.drawRect(rect.normalized())

In [8]:
def selecionar_arq(parent, modo, tipo_arquivo):
    options = QFileDialog.Options()
    options |= QFileDialog.DontUseNativeDialog
    file_dialog = QFileDialog()
    file_dialog.setDirectory(os.path.expanduser("~/C:/Users/user/Desktop/Estudos/teti/trabalhos/manipulacao_video_img/"))
    file_path, _ = file_dialog.getOpenFileName(
        parent, 
        "Selecionar Arquivo", 
        "", 
        "Todos os Arquivos (*);;Imagens (* .png * .jpg * .jpeg);;Vídeos (* .mp4 * .avi)", 
        options=options
    )
    if file_path:
        file_info = QFileInfo(file_path)
        extension = file_info.suffix().lower()
        if extension in ['png', 'jpg', 'jpeg']:
            tipo_arquivo = "Imagem"
        elif extension in ['mp4', 'avi']:
            tipo_arquivo = "Vídeo"
        else:
            print(f"Formato de arquivo não suportado: {extension}")
            return
        parent.abrir_segunda_janela(file_path, 100, 100, tipo_arquivo)  # Passar coordenadas x e y e tipo_arquivo

In [9]:
def setup_labels_tipo_arquivo(janela, fonte):
    tipo_arquivo_label = QLabel("Tipo de Arquivo", janela)
    tipo_arquivo_label.setFont(fonte)
    tipo_arquivo_label.setStyleSheet("padding: 10px;; margin: 1px;")
    tipo_arquivo_label.adjustSize()
    tipo_arquivo_label.move(200, 150)

    return tipo_arquivo_label

def setup_labels_tipo_modo(janela, fonte):
    modo_label = QLabel("Selecione Modo", janela)
    modo_label.setFont(fonte)
    modo_label.setStyleSheet("padding: 10px; margin: 0px;")
    modo_label.adjustSize()
    modo_label.move(670, 150)

    return modo_label

def setup_combobox_tipo_arquivo(janela, fonte):
    tipo_arquivo = QComboBox(janela)
    tipo_arquivo.addItems(["Imagem", "Vídeo"])
    tipo_arquivo.setFont(fonte)
    tipo_arquivo.setStyleSheet("padding: 10px; margin: 1px;")
    tipo_arquivo.setMinimumContentsLength(10)
    tipo_arquivo.setSizeAdjustPolicy(QComboBox.AdjustToContents)
    tipo_arquivo.move(200, 200)

    return tipo_arquivo
    
def setup_combobox_modo(janela, fonte):
    modo = QComboBox(janela)
    modo.addItems(["Independente", "Cascata"])
    modo.setFont(fonte)
    modo.setStyleSheet("padding: 10px; margin: 1px;")
    modo.setMinimumContentsLength(10)
    modo.setSizeAdjustPolicy(QComboBox.AdjustToContents)
    modo.move(670, 200)

    return modo

def setup_btn_avancar(janela, fonte):
    btn_avancar = QPushButton("Avançar", janela)
    btn_avancar.setFont(fonte)
    btn_avancar.setStyleSheet("background-color: #d3d3d3;")
    btn_avancar.resize(btn_avancar.sizeHint())
    btn_avancar.move(200, 600)
    btn_avancar.clicked.connect(janela.avancar)

    return btn_avancar

def setup_btn_sair(janela, fonte):
    btn_sair = QPushButton("Sair", janela)
    btn_sair.setFont(fonte)
    btn_sair.setStyleSheet("background-color: #d3d3d3; padding: 10px 45px;")
    btn_sair.resize(btn_sair.sizeHint())
    btn_sair.move(670, 600)
    btn_sair.clicked.connect(janela.close)

    return btn_sair

In [10]:
class JanelaPrincipal(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Manipulação de Imagens e Vídeos")
        self.setGeometry(100, 100, 1000, 700)
        self._setup_ui() 

    def _setup_fontes(self):
        self.label_fonte = QFont("Segoe UI", 10, QFont.Bold)
        self.combo_fonte = QFont("Segoe UI", 10)
        self.btn_fonte = QFont("Segoe UI", 12)
        
    def _setup_ui(self):
        self._centralizar_tela()  
        self._setup_fontes() 
        self.tipo_arquivo_label = setup_labels_tipo_arquivo(self, self.label_fonte)
        self.modo_label = setup_labels_tipo_modo(self, self.label_fonte)
        self.modo = setup_combobox_modo(self, self.combo_fonte)
        self.tipo_arquivo = setup_combobox_tipo_arquivo(self, self.combo_fonte)
        self.btn_avancar = setup_btn_avancar(self, self.btn_fonte)
        self.btn_sair = setup_btn_sair(self, self.btn_fonte)    

    def avancar(self):
       modo = self.modo.currentText()
       tipo_arquivo = self.tipo_arquivo.currentText()
       selecionar_arq(self, modo, tipo_arquivo)

    def abrir_segunda_janela(self, file_path, x, y, tipo_arquivo):
        self.segunda_janela = JanelaSecundaria(file_path, x, y, tipo_arquivo, self)
        self.segunda_janela.showMaximized()

    def _centralizar_tela(self): 
        qt_rectangle = self.frameGeometry()
        center_point = QDesktopWidget().availableGeometry().center()
        qt_rectangle.moveCenter(center_point)
        self.move(qt_rectangle.topLeft())

In [11]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    janela_principal = JanelaPrincipal()
    janela_principal.show()
    app.exec_()

Carregando imagem do caminho: D:/Estudos/teti/trabalhos/manipulacao_video_img/imagens/arara-azul3.jpg
Imagem carregada com sucesso.


TypeError: JanelaSecundaria._config_botoes() takes 1 positional argument but 2 were given

Carregando imagem do caminho: D:/Estudos/teti/trabalhos/manipulacao_video_img/imagens/arara-azul3.jpg
Imagem carregada com sucesso.


TypeError: JanelaSecundaria._config_botoes() takes 1 positional argument but 2 were given